# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [14]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = '../Part02/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
df_tmp = df.groupby(['Sex']).agg({'Fare': [np.mean, np.median, np.max, np.count_nonzero, lambda x: x.mode()[0]]})

In [10]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()


5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



array([[1.        , 0.28395062, 0.125     , 0.        , 0.01415106],
       [0.        , 0.48148148, 0.125     , 0.        , 0.13913574],
       [1.        , 0.33333333, 0.        , 0.        , 0.01546857],
       ...,
       [1.        , 0.        , 0.125     , 0.33333333, 0.04577135],
       [0.        , 0.33333333, 0.        , 0.        , 0.0585561 ],
       [1.        , 0.40740741, 0.        , 0.        , 0.01512699]])

# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [11]:
# 原始特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7038635542329971

In [24]:
# 新特徵 + 邏輯斯迴歸

df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()
df = pd.merge(df, df_tmp, on='Sex')
# df_tmp.head()

In [30]:
df = df.select_dtypes([int,float])
df = df.fillna(-1)
df

,Pclass,Age,SibSp,Parch,Fare,"(Fare, mean)","(Fare, median)","(Fare, amax)","(Fare, count_nonzero)","(Fare, <lambda>)"
0,3,22.0,1,0,7.2500,25.523893,10.5,512.3292,562.0,8.05
1,3,35.0,0,0,8.0500,25.523893,10.5,512.3292,562.0,8.05
2,3,-1.0,0,0,8.4583,25.523893,10.5,512.3292,562.0,8.05
3,1,54.0,0,0,51.8625,25.523893,10.5,512.3292,562.0,8.05
4,3,2.0,3,1,21.0750,25.523893,10.5,512.3292,562.0,8.05
5,3,20.0,0,0,8.0500,25.523893,10.5,512.3292,562.0,8.05
6,3,39.0,1,5,31.2750,25.523893,10.5,512.3292,562.0,8.05
7,3,2.0,4,1,29.1250,25.523893,10.5,512.3292,562.0,8.05
8,2,-1.0,0,0,13.0000,25.523893,10.5,512.3292,562.0,8.05
9,2,35.0,0,0,26.0000,25.523893,10.5,512.3292,562.0,8.05


In [29]:
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6150413134041685